In [1]:
%env CUDA_VISIBLE_DEVICES=1
# %cd GitHub/sssl/

env: CUDA_VISIBLE_DEVICES=1


In [2]:
from torchvision.datasets import CIFAR100
import torchvision.transforms as transforms
data_root = "data/"

cifar100_train = CIFAR100(
    root=data_root, train=True, download=True, transform=transforms.ToTensor()
)
cifar100_test = CIFAR100(
    root=data_root, train=False, transform=transforms.ToTensor()
)

In [8]:
import os
name = "epoch=999-step=59000.ckpt"
name2 = "epoch=999.ckpt"
log_dir = "logs/"
fully_trained_ckpt_paths = []
for (root,dirs,files) in os.walk(log_dir, topdown=True):
    if name in files:
        fully_trained_ckpt_paths.append(root+"/"+name)
    if name2 in files:
        fully_trained_ckpt_paths.append(root+"/"+name2)
fully_trained_ckpt_paths

['logs/SimCLR-l2/version_-0/checkpoints/epoch=999-step=59000.ckpt',
 'logs/SimCLR-l2/version_2/checkpoints/epoch=999-step=59000.ckpt',
 'logs/SimCLR-angle2/version_-0/checkpoints/epoch=999-step=59000.ckpt',
 'logs/SimCLR-exp_map/version_-0/checkpoints/epoch=999-step=59000.ckpt',
 'logs/SimCLR-exp_map/version_2/checkpoints/epoch=999-step=59000.ckpt',
 'logs/SimCLR-exp_map/version_-1/checkpoints/epoch=999-step=59000.ckpt',
 'logs/Comparisons/exp_map/version_1/checkpoint/epoch=999.ckpt',
 'logs/Comparisons/angle2/version_1/checkpoint/epoch=999.ckpt',
 'logs/Comparisons/l2/version_1/checkpoint/epoch=999.ckpt',
 'logs/Comparisons/stereo/version_1/checkpoint/epoch=999.ckpt',
 'logs/SimCLR-stereo/version_-0/checkpoints/epoch=999-step=59000.ckpt']

In [ ]:
from lightning_simclr import SimCLR

failed_exp_map = "logs/SimCLR-exp_map/version_-1/checkpoints/epoch=999-step=59000.ckpt"
model = SimCLR.load_from_checkpoint(failed_exp_map, strict=False)

In [ ]:
from lightning_simclr import dataset_to_X_y
from data_utils import cifar10_train, cifar10_test
import torch
train_dataset = cifar10_train
test_dataset = cifar10_test
device = model.device()

model.eval()
    
with torch.no_grad():
    X_train, y_train, Z_train = dataset_to_X_y(train_dataset, model, device)
    X_test, y_test, Z_test = dataset_to_X_y(test_dataset, model, device)

device(type='cuda', index=0)

In [ ]:
from lightning_simclr import SimCLR, eval

for ckpt_path in fully_trained_ckpt_paths:
    print(ckpt_path)
    model = SimCLR.load_from_checkpoint(ckpt_path+"/"+name)
    eval(model, ckpt_path, save_accuracies=False, train_dataset=cifar100_train, test_dataset=cifar100_test)

logs/SimCLR-l2/version_-0/checkpoints
Computing features for evaluation
kNN accuracy (Euclidean): 0.3346
kNN accuracy (cosine): 0.3442
kNN accuracy (cosine), Z: 0.2571
kNN accuracy (cosine), Z after normalization: 0.2571
logs/SimCLR-l2/version_2/checkpoints
Computing features for evaluation
kNN accuracy (Euclidean): 0.3241
kNN accuracy (cosine): 0.3420
kNN accuracy (cosine), Z: 0.2467
kNN accuracy (cosine), Z after normalization: 0.2467
logs/SimCLR-angle2/version_-0/checkpoints
Computing features for evaluation
kNN accuracy (Euclidean): 0.3279
kNN accuracy (cosine): 0.3302
kNN accuracy (cosine), Z: 0.2782
kNN accuracy (cosine), Z after normalization: 0.2883
logs/SimCLR-exp_map/version_-0/checkpoints
Computing features for evaluation
kNN accuracy (Euclidean): 0.1968
kNN accuracy (cosine): 0.1969
kNN accuracy (cosine), Z: 0.1635
kNN accuracy (cosine), Z after normalization: 0.1832
logs/SimCLR-exp_map/version_2/checkpoints
Computing features for evaluation
kNN accuracy (Euclidean): 0.3171

In [3]:
from lightning_simclr_cifar10 import SimCLR, eval

for ckpt_path in fully_trained_ckpt_paths:
    print(ckpt_path)
    model = SimCLR.load_from_checkpoint(ckpt_path+"/"+name)
    eval(model, ckpt_path, save_accuracies=False)

logs/SimCLR-l2/version_-0/checkpoints
Computing features for evaluation
kNN accuracy (Euclidean): 0.9137
kNN accuracy (cosine): 0.9162
kNN accuracy (cosine), Z: 0.9086
kNN accuracy (cosine), Z after normalization: 0.9086
logs/SimCLR-l2/version_2/checkpoints
Computing features for evaluation
kNN accuracy (Euclidean): 0.916
kNN accuracy (cosine): 0.9208
kNN accuracy (cosine), Z: 0.9125
kNN accuracy (cosine), Z after normalization: 0.9125
logs/SimCLR-angle2/version_-0/checkpoints
Computing features for evaluation
kNN accuracy (Euclidean): 0.904
kNN accuracy (cosine): 0.9025
kNN accuracy (cosine), Z: 0.8936
kNN accuracy (cosine), Z after normalization: 0.8987
logs/SimCLR-exp_map/version_-0/checkpoints
Computing features for evaluation
kNN accuracy (Euclidean): 0.8307
kNN accuracy (cosine): 0.8308
kNN accuracy (cosine), Z: 0.8200
kNN accuracy (cosine), Z after normalization: 0.8269
logs/SimCLR-exp_map/version_2/checkpoints
Computing features for evaluation
kNN accuracy (Euclidean): 0.9131
k

In [7]:
from torchvision.datasets import CIFAR10, CIFAR100, Flowers102, StanfordCars

In [ ]:
data_root = "data/"
flowers102_train = Flowers102(
    root=data_root, split="train", download=True, transform=transforms.ToTensor()
)
flowers102_test = Flowers102(
    root=data_root, split="test", download=True, transform=transforms.ToTensor()
)

100%|██████████| 345M/345M [00:08<00:00, 38.8MB/s] 
100%|██████████| 502/502 [00:00<00:00, 1.39MB/s]
100%|██████████| 15.0k/15.0k [00:00<00:00, 57.4MB/s]


In [20]:
flowers102_test

Dataset Flowers102
    Number of datapoints: 6149
    Root location: data/
    split=test
    StandardTransform
Transform: ToTensor()